In [3]:
from analysis import *
import dashboard

In [4]:
def check_model_results(data, period = 'D', d = 0, suffix=None, graph=False):
    if period == 'D':
        num_lags = 40
    elif period == 'W':
        num_lags = 14
    elif period == 'M':
        num_lags = 7
    else:
        return 0
    FAC = {}
    FAP = {}
    data.reset_index(inplace=True)
    data['Data'] = pd.to_datetime(data['Data'])
    data.set_index('Data', inplace=True)
    res = []
    for cg in data.CityGate.unique():
        teste = data[data['CityGate'] == cg].resample(period).sum()
        if graph:
            fig, ax = plt.subplots(3,2,figsize=(24, 16))
            fig.suptitle(cg, fontsize=16)
            lista_fac = sm.tsa.acf(teste.Difference,
                               nlags = num_lags,
                               fft = False)
            FAC.update({cg: lista_fac})
            lista_fap = sm.tsa.pacf(teste.Difference,
                                   nlags = num_lags)
            FAP.update({cg: lista_fap})
            sns.lineplot(data=teste, x=teste.index, y= "Volume_dia", ax=ax[0][0])
            sns.lineplot(data=teste, x=teste.index, y = "Volume_Previsto_Dia", ax=ax[0][0])
            sm.graphics.tsa.plot_acf(teste['Difference'], lags = num_lags, alpha = 0.05, ax=ax[0][1])
            sm.graphics.tsa.plot_pacf(teste['Difference'], lags =num_lags,alpha = 0.05, ax=ax[1][1])
            sns.lineplot(data=teste, x=teste.index, y="Difference", ax=ax[1][0])
            sns.histplot(data=teste, x="Difference",  ax=ax[2][0])
            alpha = 0.01
            mape = fr'mape = {round(teste.Absolute_Percentage_Error.mean(),3)}%'
            acima5 = fr"dias acima de 5% = {teste['Acima5%'].sum()} ({round(teste['Acima5%'].mean() * 100, 3)}%)"
            min_max = fr"Min: {round(teste.Absolute_Percentage_Error.min(),3)}% |  Max: {round(teste.Absolute_Percentage_Error.max(),3)}%"
            ax[2][1].text(0.5,0.8, mape, horizontalalignment='center',fontsize = 25,
                          verticalalignment='center', transform = ax[2][1].transAxes)
            ax[2][1].text(0.5,0.6, acima5, horizontalalignment='center',fontsize = 25,
                          verticalalignment='center', transform = ax[2][1].transAxes)    
            ax[2][1].text(0.5,0.4, min_max, horizontalalignment='center',fontsize = 25,
                          verticalalignment='center', transform = ax[2][1].transAxes)  
            ax[2][1].grid(False)
            plt.show()
        mape = round(teste.Absolute_Percentage_Error.mean(),3)
        acima5c = teste['Acima5%'].sum()
        acima5p = round(teste['Acima5%'].mean() * 100, 3)
        minimo = round(teste.Absolute_Percentage_Error.min(),3)
        maximo = round(teste.Absolute_Percentage_Error.max(),3)
        res.append([cg, mape, acima5c, acima5p, minimo, maximo])
    cols = ['CityGate', 'MAPE', 'Acima5_dias', 'Acima5_percent', 'Minimo', 'Maximo']
    if suffix is not None:
        cols = [f"{c}_{suffix}" if c != 'CityGate' else c for c in cols]
    res = pd.DataFrame(res, columns=cols)
    return res
